In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[11]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.0,0.000000,5.584,0.0,0.0
1,5123.9,0.586782,5.219,0.0,0.0
2,4771.1,0.546380,5.034,0.0,0.0
3,4443.5,0.508864,4.708,0.0,0.0
4,4234.9,0.484975,4.353,0.0,0.0
...,...,...,...,...,...
43819,6788.8,0.777444,9.646,0.0,0.0
43820,6193.0,0.709214,9.172,0.0,0.0
43821,5706.9,0.653547,8.705,0.0,0.0
43822,5484.7,0.628101,8.218,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[11]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_PT,Temp_PT
0,0.0,5.584
1,5123.9,5.219
2,4771.1,5.034
3,4443.5,4.708
4,4234.9,4.353
...,...,...
43819,6788.8,9.646
43820,6193.0,9.172
43821,5706.9,8.705
43822,5484.7,8.218


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_PT,Temp_PT,Lag-2,Lag-7
0,6047.2,4.941,5740.7,0.0
1,5531.3,4.527,5270.9,5123.9
2,5255.4,4.240,4964.8,4771.1
3,5121.0,3.992,4851.3,4443.5
4,5076.9,3.816,4818.9,4234.9
...,...,...,...,...
43651,6788.8,9.646,6455.6,6030.4
43652,6193.0,9.172,6411.9,5400.7
43653,5706.9,8.705,6226.7,4945.0
43654,5484.7,8.218,5857.5,4814.1


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 33s 16ms/step - loss: 0.0188 - mape: 7138.0752 - mae: 0.1079 - val_loss: 0.0350 - val_mape: 34.0531 - val_mae: 0.1487
Epoch 2/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0081 - mape: 4164.2754 - mae: 0.0671 - val_loss: 0.0185 - val_mape: 27.6355 - val_mae: 0.1139
Epoch 3/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0069 - mape: 4144.7896 - mae: 0.0605 - val_loss: 0.0171 - val_mape: 26.2151 - val_mae: 0.1084
Epoch 4/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0063 - mape: 3957.2505 - mae: 0.0575 - val_loss: 0.0294 - val_mape: 35.8883 - val_mae: 0.1462
Epoch 5/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0058 - mape: 3479.3821 - mae: 0.0548 - val_loss: 0.0379 - val_mape: 42.5247 - val_mae: 0.1721
Epoch 6/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0055 - mape: 3786.9263 - mae: 0.0529 - val_l

1105/1105 [==============================] - 17s 15ms/step - loss: 0.0026 - mape: 1954.9202 - mae: 0.0344 - val_loss: 0.0205 - val_mape: 32.0241 - val_mae: 0.1189
Epoch 48/100
1105/1105 [==============================] - 16s 15ms/step - loss: 0.0025 - mape: 2110.6982 - mae: 0.0341 - val_loss: 0.0207 - val_mape: 30.9958 - val_mae: 0.1203
Epoch 49/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0025 - mape: 2356.5078 - mae: 0.0343 - val_loss: 0.0217 - val_mape: 32.4165 - val_mae: 0.1217
Epoch 50/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0025 - mape: 2221.5466 - mae: 0.0337 - val_loss: 0.0242 - val_mape: 36.1823 - val_mae: 0.1328
Epoch 51/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0024 - mape: 2030.7837 - mae: 0.0334 - val_loss: 0.0230 - val_mape: 33.0908 - val_mae: 0.1242
Epoch 52/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0024 - mape: 2151.8254 - mae: 0.0333 - val_loss: 0.

1105/1105 [==============================] - 18s 16ms/step - loss: 0.0018 - mape: 1348.6586 - mae: 0.0288 - val_loss: 0.0228 - val_mape: 32.8775 - val_mae: 0.1249
Epoch 94/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0015 - mape: 1508.9277 - mae: 0.0269 - val_loss: 0.0233 - val_mape: 33.5043 - val_mae: 0.1237
Epoch 95/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0015 - mape: 1316.1479 - mae: 0.0272 - val_loss: 0.0220 - val_mape: 31.8096 - val_mae: 0.1213
Epoch 96/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0015 - mape: 1006.3196 - mae: 0.0271 - val_loss: 0.0269 - val_mape: 36.0682 - val_mae: 0.1378
Epoch 97/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0015 - mape: 1268.5441 - mae: 0.0272 - val_loss: 0.0225 - val_mape: 30.7708 - val_mae: 0.1200
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0015 - mape: 1197.4901 - mae: 0.0270 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1772.6559019088745


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 13ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.17808601, 0.13874334, 0.11625142, ..., 0.36693567, 0.31898713,
       0.26923972], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.3081441 , 0.27428828, 0.25988553, ..., 0.43412143, 0.3925592 ,
       0.34749916])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,5033.4,4338.083496
1,4852.4,4127.749512
2,4775.4,4007.503418
3,4733.0,3960.605957
4,4740.7,3857.183105
...,...,...
4363,6788.8,6131.937988
4364,6193.0,5770.743652
4365,5706.9,5347.711914
4366,5484.7,5091.369141


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.848128350329985